In [26]:
import requests
from bs4 import BeautifulSoup
import re

In [42]:
class Crawler():
    def __init__(self, url, max_pages):
        self.url = url
        self.max_pages = max_pages

    def get_url(self, current_url, urls_found):
        urls = set([])
        try:
            response = requests.get(current_url)
        except:
            print('Unable to connect to internet or Invalid link. Please check')
            return urls
        soup = BeautifulSoup(response.text, "lxml")
        for i in soup(['script', 'style']):
            i.extract()
            
        with open(f"{self.algorithm}/{abs(hash(current_url))}.txt", "w") as f:
            f.write(soup.get_text())
            
        newLink = soup.findAll('a')
        for link in newLink:
            try:
                link = str(link.get('href'))
                m = re.match(r'http+', link)
                if not m:
                    link = current_url + link
                if link not in urls_found:
                    urls.add(str(link))
            except UnicodeEncodeError as e:
                print('Error ---> ', e)
        return urls
        
    def start(self, algorithm):
        self.algorithm = algorithm
        structure = []
        urls_found = set([])
        structure.append(self.url)
        count = 0
        while count < self.max_pages and structure:
            if algorithm == "bfs":
                current_url = structure.pop(0)
            else:
                current_url = structure.pop()
            urls = self.get_url(current_url, urls_found)
            print('[%s] links found in %s' %(len(urls), current_url))
            for i in urls:
                structure.append(i)
                urls_found.add(i)
            count += 1
        return urls_found
        
    def __str__(self):
        return 'url:%s' % (self.url)

In [48]:
crawler = Crawler("https://www.dreamhost.com/", 5)

In [49]:
_ = crawler.start("bfs")

[57] links found in https://www.dreamhost.com/
[89] links found in https://www.dreamhost.com//press/
[29] links found in https://www.dreamhost.com///www.dreamhost.com
[39] links found in https://www.dreamhost.com//hosting/vps/
[38] links found in https://www.dreamhost.com//hosting/dedicated/


In [50]:
_ = crawler.start("dfs")

[57] links found in https://www.dreamhost.com/
[38] links found in http://www.dreamhost.com/hosting/dedicated/
[29] links found in http://www.dreamhost.com/hosting/dedicated//hosting/shared/
[28] links found in http://www.dreamhost.com/hosting/dedicated//hosting/shared//blog
[28] links found in http://www.dreamhost.com/hosting/dedicated//hosting/shared//blog//iad2.dreamcompute.com
